In [1]:
import syft as sy
import numpy as np
from sqlalchemy import create_engine
from syft.core.node.common.node_table import Base
from nacl.encoding import HexEncoder
from nacl.signing import SigningKey
from syft.core.adp.scalar.gamma_scalar import GammaScalar
from syft.core.tensor.autodp.initial_gamma import InitialGammaTensor
from syft.core.tensor.autodp.intermediate_gamma import IntermediateGammaTensor
from syft.core.adp.entity import Entity
from syft.core.adp.entity import DataSubjectGroup
from syft.core.adp.adversarial_accountant import AdversarialAccountant
from syft.core.io.virtual import create_virtual_connection

In [2]:
# Initialize domain node
alice = sy.Domain("alice").initial_setup()

Setup hasnt run yet so ignoring set_node_uid
Performing initial setup...


In [3]:
# Data
child = np.array([1,2,3,4], dtype=np.int32)

In [4]:
# Entities
traskmaster = Entity(name="Andrew")
amber = Entity(name="Amber")
DSG = DataSubjectGroup([traskmaster, amber])

ent1 = np.array([traskmaster, traskmaster, traskmaster, traskmaster], dtype=object)
ent2 = np.array([amber, amber, amber, amber], dtype=object)

In [5]:
# gs = GammaScalar(min_val=0, max_val=5, value=3, prime=13, entity=traskmaster)
tensor1 = InitialGammaTensor(values=child, min_vals=np.zeros_like(child), max_vals=np.ones_like(child) * 5, entities=ent1)
tensor2 = InitialGammaTensor(values=child, min_vals=np.zeros_like(child), max_vals=np.ones_like(child)* 5, entities=ent2)

In [6]:
tensor1.publish(sigma=50, acc=alice.acc, user_key=alice.verify_key)



Has Budget:True
YOU'VE SPENT:0.8999352676606419
USER BUDGET:5.55


/home/e/.local/lib/python3.8/site-packages/scipy/optimize/optimize.py:2621: RuntimeWarning: invalid value encountered in double_scalars
  w = xb - ((xb - xc) * tmp2 - (xb - xa) * tmp1) / denom


array([-61.10096114,  62.53936008,  15.55809723, -38.47603157])

In [7]:
alice.acc.print_ledger()

Andrew	0.8999352676606419


In [9]:
# Create a 1D IntermediateGammaTensor of just DSGs
dsg_ent = np.array([DSG, DSG, DSG, DSG], dtype=object)
tensor1 = InitialGammaTensor(values=child, min_vals=np.zeros_like(child), max_vals=np.ones_like(child) * 5, entities=dsg_ent)
IGT = tensor1 + tensor1

In [10]:
assert IGT._values().shape == IGT._entities().shape

array([2., 4., 6., 8.])

In [12]:
entities = IGT._entities()

print(len(entities))
for i, e in enumerate(entities):
    print(f'index = {i} of type = {type(e)} with value = {e}')

(4,)

In [16]:
np.array(IGT._entities()).reshape(IGT.shape)

array([DSG['<Entity:Amber>', '<Entity:Andrew>'],
       DSG['<Entity:Amber>', '<Entity:Andrew>'],
       DSG['<Entity:Amber>', '<Entity:Andrew>'],
       DSG['<Entity:Amber>', '<Entity:Andrew>']], dtype=object)

In [17]:
IGT.flat_scalars[0].input_scalars[0].entity

DSG['<Entity:Amber>', '<Entity:Andrew>']

In [18]:
IGT.flat_scalars

[<IntermediateGammaScalar: (0.0 < 2.0 < 10.0)>,
 <IntermediateGammaScalar: (0.0 < 4.0 < 10.0)>,
 <IntermediateGammaScalar: (0.0 < 6.0 < 10.0)>,
 <IntermediateGammaScalar: (0.0 < 8.0 < 10.0)>]

In [19]:
tensor1.publish(sigma=50, acc=alice.acc, user_key=alice.verify_key)



Has Budget:True
YOU'VE SPENT:0.0
USER BUDGET:5.55


Has Budget:True
YOU'VE SPENT:0.8999352676606419
USER BUDGET:5.55


array([ 68.47188461, -32.9585616 , -37.93729601,  57.51212366])

In [20]:
alice.acc.print_ledger()

Andrew	1.3049911193525312
Amber	0.8999352676606419


In [21]:
alice.acc.print_ledger()

Andrew	1.3049911193525312
Amber	0.8999352676606419


In [22]:
alice.acc.entity2ledger.mechanism_manager.all()

In [23]:
# conn = create_virtual_connection(alice)